#                          LOENTIEF INPUT OUTPUT MODEL
The input-output model is an economic tool used to analyze interdependencies among different sectors within an economy. It quantifies the relationships between sectors by capturing the flows of goods, services, and payments between them. 

In this program we will be checking the expected output for each sector, given their input values.

# STEPS TO CALCULATE FOR EXPECTED OUTPUT
To calculate for the expected outputs of each sector, we follow the following steps



### Step 1: Import the neccessary libraries

In [2]:
import numpy as np
import pandas as pd
from scipy.linalg import pinv

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### Step 2: Import the CSV file.(This file has different sectors and their input as stated by [BEA Table 6.6B](https://apps.bea.gov/iTable/?reqid=150&step=3&isuri=1&table_list=6006&categories=io&_gl=1*1gckkur*_ga*NDUzMTk3OTQ4LjE3MDk0MTA2MTA.*_ga_J4698JNNFT*MTcwOTQxMDYwOS4xLjEuMTcwOTQxMTI2Ny42MC4wLjA.#eyJhcHBpZCI6MTUwLCJzdGVwcyI6WzEsMiwzXSwiZGF0YSI6W1sidGFibGVfbGlzdCIsIjYwMDkiXSxbImNhdGVnb3JpZXMiLCJHZHB4SW5kIl1dfQ==)

The table gives the amount given as output from one sector and used by the other sector as output. To further explain with an example, the sector; 'Farm' gave $4097 which in turn is used as output by the sector;'Construction'

In [153]:
df = pd.read_csv('C:/Users/dell/Desktop/PROJECTS/Leontief2/Table.csv')
df.head(2)

,Name,"Agriculture, forestry, fishing, and hunting",Mining,Utilities,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,"Finance, insurance, real estate, rental, and leasing",Professional and business services,"Educational services, health care, and social assistance","Arts, entertainment, recreation, accommodation, and food services","Other services, except government",Government,Total Commodity Output,Imports
0,"Agriculture, forestry, fishing, and hunting",641691,0,0,0,0,95,274,143,0,0,0,0,0,0,5663,649153,82128
1,Mining,0,74119,14,0,746,0,0,0,0,0,335,0,0,0,836,742176,228268


In [16]:
df.shape

(17, 18)

# Creating the coefficient matrix for this table
To find the coefficient values, for each sector, divide the value by the equivalent row total. To achieve this, we have to add a Total column for each row.

In [154]:
# Calculate row totals
df['Total']= df.sum(axis=1)
df.head(2)

,Name,"Agriculture, forestry, fishing, and hunting",Mining,Utilities,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,"Finance, insurance, real estate, rental, and leasing",Professional and business services,"Educational services, health care, and social assistance","Arts, entertainment, recreation, accommodation, and food services","Other services, except government",Government,Total Commodity Output,Imports,Total
0,"Agriculture, forestry, fishing, and hunting",641691,0,0,0,0,95,274,143,0,0,0,0,0,0,5663,649153,82128,1379052
1,Mining,0,74119,14,0,746,0,0,0,0,0,335,0,0,0,836,742176,228268,1046494


In [124]:
df.shape

(17, 19)

In [155]:
# Extract the columns to divide (excluding 'Name' and 'Total' columns)
columns_to_divide = df.columns[1:-1]

# Convert columns to numeric type
numeric_cols = df.columns.drop(['Name', 'Total'])  # Exclude 'Name' and 'Total'
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Create a new DataFrame for the coefficient matrix
coefficient_matrix = pd.DataFrame(index=df.index, columns=columns_to_divide)

# Perform the division operation
for idx, column in enumerate(columns_to_divide):
    coefficient_matrix[column] = df[column] / df['Total'].iloc[idx]

pd.DataFrame(coefficient_matrix)[:3]

,"Agriculture, forestry, fishing, and hunting",Mining,Utilities,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,"Finance, insurance, real estate, rental, and leasing",Professional and business services,"Educational services, health care, and social assistance","Arts, entertainment, recreation, accommodation, and food services","Other services, except government",Government,Total Commodity Output,Imports
0,0.465313,0.000000,0.000000,0.0,0.000000,0.000038,0.000111,0.000254,0.0,0.0,0.000000,0.0,0.0,0.0,0.000750,17.001545,2.037511
1,0.000000,0.070826,0.000019,0.0,0.000094,0.000000,0.000000,0.000000,0.0,0.0,0.000048,0.0,0.0,0.0,0.000111,19.437850,5.663094
2,0.000000,0.000000,0.881146,0.0,0.000000,0.000000,0.000000,0.000403,0.0,0.0,0.000000,0.0,0.0,0.0,0.000264,2.218925,0.011363


# CREATING THE IDENTITY MATRIX
The formula for solving the input output problem is X = (I- A)^-1 D

where I = Identity Matrix

      A = Coefficient Matrix  
      
      D = Final Demand

In [156]:
# Check the shape of the coefficient_matrix
coefficient_matrix.shape

(17, 17)

### Identity_matrix

We are going to find the identity matrix according to the shape of the coefficient matrix so we can carry out our subtraction


In [157]:
# Get the shape of the DataFrame
df_shape = coefficient_matrix.shape

# Create an identity matrix with the same shape as the DataFrame
identity_matrix = np.eye(df_shape[0], df_shape[1])

# Convert the identity matrix to a DataFrame
identity_df = pd.DataFrame(identity_matrix,index=coefficient_matrix.index, columns=coefficient_matrix.columns)
identity_df.head(2)

,"Agriculture, forestry, fishing, and hunting",Mining,Utilities,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,"Finance, insurance, real estate, rental, and leasing",Professional and business services,"Educational services, health care, and social assistance","Arts, entertainment, recreation, accommodation, and food services","Other services, except government",Government,Total Commodity Output,Imports
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
identity_df.shape

(17, 17)

# Calculating for I - A
We subtract the coefficient matrix from the identity matrix, we perform the basic subtraction

In [158]:
I = identity_df.to_numpy()
A = coefficient_matrix.to_numpy()
Q = I - A
difference = pd.DataFrame(Q)
difference.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.534687,0.000000,0.000000,0.0,0.000000,-0.000038,-0.000111,-0.000254,0.0,0.0,0.000000,0.0,0.0,0.0,-0.000750,-17.001545,-2.037511
1,0.000000,0.929174,-0.000019,0.0,-0.000094,0.000000,0.000000,0.000000,0.0,0.0,-0.000048,0.0,0.0,0.0,-0.000111,-19.437850,-5.663094


# Finding the Inverse of I - A

In [170]:
# Convert DataFrame to NumPy array
array = difference.to_numpy()

# Calculate the determinant of the array
determinant = np.linalg.det(array)
inverse = np.linalg.inv(array)

print(f'inverse is: {inverse}')
print("")
print(f' The determinant is: {determinant}')

inverse is: [[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan